# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-14-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-14-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-15 04:27:31,33.939110,67.709953,37431,1363,26714,9354.0,Afghanistan,96.153597,3.641367
1,NaN,NaN,NaN,Albania,2020-08-15 04:27:31,41.153300,20.168300,7117,219,3695,3203.0,Albania,247.306971,3.077139
2,NaN,NaN,NaN,Algeria,2020-08-15 04:27:31,28.033900,1.659600,37664,1351,26308,10005.0,Algeria,85.890773,3.586980
3,NaN,NaN,NaN,Andorra,2020-08-15 04:27:31,42.506300,1.521800,989,53,863,73.0,Andorra,1280.010354,5.358948
4,NaN,NaN,NaN,Angola,2020-08-15 04:27:31,-11.202700,17.873900,1852,86,584,1182.0,Angola,5.634957,4.643629
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-15 04:27:31,17.060800,-61.796400,93,3,83,7.0,Antigua and Barbuda,94.967731,3.225806
6,NaN,NaN,NaN,Argentina,2020-08-15 04:27:31,-38.416100,-63.616700,282437,5527,199005,77905.0,Argentina,624.919005,1.956897
7,NaN,NaN,NaN,Armenia,2020-08-15 04:27:31,40.069100,45.038200,41299,814,34164,6321.0,Armenia,1393.713760,1.970992
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-15 04:27:31,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-15 04:27:31,-33.868800,151.209300,3945,51,2994,900.0,"New South Wales, Australia",48.595713,1.292776


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36829,36896,37015,37054,37054,37162,37269,37345,37424,37431
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5889,6016,6151,6275,6411,6536,6676,6817,6971,7117
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,33055,33626,34155,34693,35160,35712,36204,36699,37187,37664


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1294,1298,1307,1312,1312,1328,1344,1354,1363,1363
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,182,188,189,193,199,200,205,208,213,219
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1261,1273,1282,1293,1302,1312,1322,1333,1341,1351


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25742,25840,25903,25960,25960,26228,26415,26694,26714,26714
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3123,3155,3227,3268,3342,3379,3480,3552,3616,3695
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,22802,23238,23667,24083,24506,24920,25263,25627,26004,26308


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1030,1050,1065,1086,1169,1174,1162,1188,1196,1196
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3322,3380,3443,3502,3544,3651,3670,3710,3744,3791
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,564,565,569,573,575,576,579,581,586,590


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,21,21,21,21,21,21,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,23,23,23,24,25,25,29,29,29,29
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,5,5,5,5,5,6,6,6,6


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-15 04:27:31,32.539527,-86.644082,1196,22,0,1174.0,"Autauga, Alabama, US",2140.722046,1.839465
668,1075.0,Lamar,Alabama,US,2020-08-15 04:27:31,33.779950,-88.096680,237,2,0,235.0,"Lamar, Alabama, US",1716.769286,0.843882
669,1077.0,Lauderdale,Alabama,US,2020-08-15 04:27:31,34.901719,-87.656247,1219,20,0,1199.0,"Lauderdale, Alabama, US",1314.583356,1.640689


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5313055,168446,1796326
Brazil,3226443,105490,2616981
India,2525922,49036,1808936


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5313055,168446,1796326,3348283,2020-08-15 04:27:31,37.913308,-91.347527
Brazil,3226443,105490,2616981,503972,2020-08-15 04:27:31,-12.669522,-48.480493
India,2525922,49036,1808936,667950,2020-08-15 04:27:31,23.088275,81.806127
Russia,910778,15467,721473,173838,2020-08-15 04:27:31,54.546312,62.120860
South Africa,579140,11556,461734,105850,2020-08-15 04:27:31,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,613101,11148,0
Florida,563285,9141,0
Texas,539673,10078,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,613101,11148,0,601953,2020-08-15 04:27:31,37.843962,-120.728594
Florida,563285,9141,0,554144,2020-08-15 04:27:31,28.940755,-82.700744
Texas,539673,10078,0,529595,2020-08-15 04:27:31,31.660643,-98.653069
New York,424167,32827,0,391340,2020-08-15 04:27:31,42.671593,-75.579694
Georgia,231895,4573,0,227322,2020-08-15 04:27:31,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,229534,23615,0
California,Los Angeles,218831,5215,0
Florida,Miami-Dade,142662,1999,0
Arizona,Maricopa,128352,2536,0
Illinois,Cook,114724,4951,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,229534,23615,0,205919,2020-08-15 04:27:31,40.767273,-73.971526,36061.0
California,Los Angeles,218831,5215,0,213616,2020-08-15 04:27:31,34.308284,-118.228241,6037.0
Florida,Miami-Dade,142662,1999,0,140663,2020-08-15 04:27:31,25.611236,-80.551706,12086.0
Arizona,Maricopa,128352,2536,0,125816,2020-08-15 04:27:31,33.348359,-112.491815,4013.0
Illinois,Cook,114724,4951,0,109773,2020-08-15 04:27:31,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,37345,6817,36699,977,1762,92,268574,40794,22358,22439,...,315581,1393,32654,29088,883,15184,10,1841,8501,4893
2020-08-13,37424,6971,37187,981,1815,92,276072,41023,22742,22594,...,316729,1409,33323,30369,911,15491,10,1847,8663,4990
2020-08-14,37431,7117,37664,989,1852,93,282437,41299,23035,22876,...,315621,1421,33821,31381,930,15834,10,1858,9021,5072


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,1354,208,1333,53,80,3,5213,806,361,724,...,46791,37,211,247,18,105,1,528,246,122
2020-08-13,1363,213,1341,53,80,3,5362,809,375,725,...,46791,37,216,259,21,106,1,528,246,128
2020-08-14,1363,219,1351,53,86,3,5527,814,379,725,...,46791,38,220,266,22,106,1,528,256,128


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,26694,3552,25627,855,577,76,187283,33492,12779,20268,...,1472,1163,25659,21042,409,8369,8,937,7233,1620
2020-08-13,26714,3616,26004,858,577,76,192434,33897,13001,20346,...,1479,1180,27213,21385,425,9186,8,949,7401,1927
2020-08-14,26714,3695,26308,863,584,83,199005,34164,13632,20499,...,1481,1182,27825,21580,437,9382,8,1009,7586,1998


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1030,1050,1065,1086,1169,1174,1162,1188,1196,1196
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,205,212,217,220,222,229,230,230,237,237
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1111,1123,1139,1157,1167,1189,1190,1203,1213,1219
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,325,332,344,348,349,351,354,356,360,364
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2554,2588,2628,2660,2678,2700,2708,2724,2736,2735


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,104786,3879,0,189443,51096,591376,51737,50706,15765,49,...,126393,523977,45090,1478,639,102521,64702,8018,62263,3086
2020-08-13,105557,3963,0,190794,51766,597984,52200,50782,15967,49,...,128511,530857,45424,1484,682,103622,65339,8163,63206,3119
2020-08-14,106309,4073,0,191721,52392,613101,52519,50897,16340,49,...,130458,539673,45976,1501,704,104838,66139,8277,64227,3183


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-12        1188    3710     581  453    825     492    773    1828   
2020-08-13        1196    3744     586  457    835     493    775    1853   
2020-08-14        1196    3791     590  465    848     494    779    1867   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-12          848      279  ...     135      5       73       40   
2020-08-13          848      288  ...     139      6       74       40   
2020-08-14          852      293  ...     144      6       78       40   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-12            266   373   278          0       82      5  
2020-08-13            267   375   278          0       86      5  
2020-08-14            271   383   277          0       96      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,1882,27,0,4347,572,10753,1875,4450,592,0,...,1289,9527,351,58,9,2352,1724,153,1011,29
2020-08-13,1890,27,0,4383,582,10870,1882,4450,593,0,...,1313,9771,353,58,9,2363,1736,153,1018,30
2020-08-14,1893,27,0,4423,587,11148,1888,4453,593,0,...,1326,10078,360,58,9,2370,1755,157,1025,30


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-12          22      29       6    5      5      11     36      19   
2020-08-13          22      29       6    5      5      11     36      19   
2020-08-14          22      29       6    5      5      11     36      19   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-12           38        8  ...       0      0        0        0   
2020-08-13           38        8  ...       0      0        0        0   
2020-08-14           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-12              0     0     0         28        0      0  
2020-08-13              0     0     0         29        0      0  
2020-08-14              0     0     0         29        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.002039,0.021120,0.013673,0.014538,0.015562,0.00000,0.029370,0.004952,0.010440,0.008721,...,0.003303,0.005776,0.028570,0.041163,0.019630,0.020773,0.0,0.005461,0.027311,0.015567
2020-08-13,0.002115,0.022591,0.013297,0.004094,0.030079,0.00000,0.027918,0.005614,0.017175,0.006908,...,0.003638,0.011486,0.020488,0.044039,0.031710,0.020219,0.0,0.003259,0.019057,0.019824
2020-08-14,0.000187,0.020944,0.012827,0.008155,0.020386,0.01087,0.023056,0.006728,0.012884,0.012481,...,-0.003498,0.008517,0.014945,0.033323,0.020856,0.022142,0.0,0.005956,0.041325,0.016433


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.007440,0.014634,0.008321,0.019231,0.000,0.0,0.041767,0.003736,0.025568,0.001383,...,0.001648,0.000000,0.034314,0.037815,0.125000,0.009615,0.0,0.00956,0.020747,0.173077
2020-08-13,0.006647,0.024038,0.006002,0.000000,0.000,0.0,0.028582,0.003722,0.038781,0.001381,...,0.000000,0.000000,0.023697,0.048583,0.166667,0.009524,0.0,0.00000,0.000000,0.049180
2020-08-14,0.000000,0.028169,0.007457,0.000000,0.075,0.0,0.030772,0.006180,0.010667,0.000000,...,0.000000,0.027027,0.018519,0.027027,0.047619,0.000000,0.0,0.00000,0.040650,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.010562,0.020690,0.014408,0.019070,0.003478,0.000000,0.032494,0.010103,0.030980,0.007206,...,0.005464,0.005186,0.082476,0.067797,-0.093126,0.022980,0.0,0.024044,0.032696,0.062992
2020-08-13,0.000749,0.018018,0.014711,0.003509,0.000000,0.000000,0.027504,0.012092,0.017372,0.003848,...,0.004755,0.014617,0.060564,0.016301,0.039120,0.097622,0.0,0.012807,0.023227,0.189506
2020-08-14,0.000000,0.021847,0.011691,0.005828,0.012132,0.092105,0.034147,0.007877,0.048535,0.007520,...,0.001352,0.001695,0.022489,0.009119,0.028235,0.021337,0.0,0.063224,0.024997,0.036845


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,0.009003,0.015711,NaN,0.003741,0.013588,0.021470,0.006126,0.000434,0.004204,0.0,...,0.011832,0.012125,0.007553,0.004076,0.109375,0.007627,0.008589,0.017642,0.007737,0.004230
2020-08-13,0.007358,0.021655,NaN,0.007131,0.013113,0.011174,0.008949,0.001499,0.012813,0.0,...,0.016757,0.013130,0.007407,0.004060,0.067293,0.010739,0.009845,0.018084,0.015145,0.010693
2020-08-14,0.007124,0.027757,NaN,0.004859,0.012093,0.025280,0.006111,0.002265,0.023361,0.0,...,0.015150,0.016607,0.012152,0.011456,0.032258,0.011735,0.012244,0.013965,0.016154,0.020519


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-12      0.022375  0.010899  0.003454  0.020270  0.011029  0.022869   
2020-08-13      0.006734  0.009164  0.008606  0.008830  0.012121  0.002033   
2020-08-14      0.000000  0.012553  0.006826  0.017505  0.015569  0.002028   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-12      0.003896  0.004948  0.001181  0.010870  ...  0.000000    0.0   
2020-08-13      0.002587  0.013676  0.000000  0.032258  ...  0.029630    0.2   
2020-08-14      0.005161  0.007555  0.004717  0.017361  ...  0.035971    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-12      0.000000  0.025641  -0.003745  0.002688 -0.003584        NaN   
2020-08-13      0.013699  0.000000   0.003759  0.005362  0.000000        NaN   
2020-08-14      0.054054  0.000000   0.014981  0.021333 -0.003597        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-12      0.037975    0.0  
2020-08-13      0.048780    0.0  
2020-08-14      0.116279    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,0.018950,0.038462,NaN,0.035246,0.010601,0.021857,0.000534,0.001350,0.001692,NaN,...,0.014162,0.033073,0.005731,0.0,0.0,0.003413,0.004662,0.040816,0.004970,0.000000
2020-08-13,0.004251,0.000000,NaN,0.008282,0.017483,0.010881,0.003733,0.000000,0.001689,NaN,...,0.018619,0.025611,0.005698,0.0,0.0,0.004677,0.006961,0.000000,0.006924,0.034483
2020-08-14,0.001587,0.000000,NaN,0.009126,0.008591,0.025575,0.003188,0.000674,0.000000,NaN,...,0.009901,0.031420,0.019830,0.0,0.0,0.002962,0.010945,0.026144,0.006876,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-08-12         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.055556   
2020-08-13         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-08-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-12          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-13          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-14          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-12            NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-13            NaN   NaN   NaN   0.035714      NaN    NaN  
2020-08-14            NaN   NaN   NaN   0.000000      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.002210,0.021035,0.014089,0.008558,0.022721,0.000013,0.028407,0.004200,0.014035,0.006970,...,0.003287,0.009022,0.024110,0.040384,0.021060,0.021798,2.467162e-17,0.005103,0.021875,0.016825
2020-08-13,0.002163,0.021813,0.013693,0.006326,0.026400,0.000006,0.028163,0.004907,0.015605,0.006939,...,0.003463,0.010254,0.022299,0.042212,0.026385,0.021009,1.233581e-17,0.004181,0.020466,0.018325
2020-08-14,0.001175,0.021378,0.013260,0.007240,0.023393,0.005438,0.025609,0.005817,0.014244,0.009710,...,-0.000018,0.009385,0.018622,0.037768,0.023621,0.021575,6.167906e-18,0.005068,0.030895,0.017379


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.008533,0.017381,0.007983,0.009615,0.018572,7.581939e-37,0.040843,0.005823,0.042950,0.001106,...,0.001350,0.000068,0.030430,0.036524,0.119463,0.020788,0.0,0.008463,0.025123,0.091633
2020-08-13,0.007590,0.020710,0.006992,0.004808,0.009286,3.790969e-37,0.034713,0.004773,0.040865,0.001244,...,0.000675,0.000034,0.027064,0.042553,0.143068,0.015156,0.0,0.004231,0.012562,0.070407
2020-08-14,0.003795,0.024439,0.007225,0.002404,0.042143,1.895485e-37,0.032742,0.005476,0.025766,0.000622,...,0.000338,0.013531,0.022791,0.034790,0.095340,0.007578,0.0,0.002116,0.026606,0.035203


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-12,0.008508,0.021588,0.015002,0.009798,0.007452,0.000170,0.187693,0.010700,0.025198,0.006296,...,0.004077,0.009008,0.060164,0.110950,-0.012315,0.022930,1.445603e-19,0.012940,0.031326,0.043213
2020-08-13,0.004629,0.019803,0.014856,0.006653,0.003726,0.000085,0.107598,0.011396,0.021285,0.005072,...,0.004416,0.011813,0.060364,0.063625,0.013402,0.060276,7.228014e-20,0.012874,0.027276,0.116360
2020-08-14,0.002314,0.020825,0.013273,0.006240,0.007929,0.046095,0.070873,0.009637,0.034910,0.006296,...,0.002884,0.006754,0.041427,0.036372,0.020819,0.040806,3.614007e-20,0.038049,0.026136,0.076602


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,0.010450,0.015806,NaN,0.004589,0.012181,0.017463,0.006949,0.001452,0.004205,4.770378e-46,...,0.011334,0.013232,0.007721,0.004454,0.074096,0.008573,0.008810,0.015934,0.009641,0.005652
2020-08-13,0.008904,0.018730,NaN,0.005860,0.012647,0.014319,0.007949,0.001475,0.008509,2.385189e-46,...,0.014046,0.013181,0.007564,0.004257,0.070695,0.009656,0.009328,0.017009,0.012393,0.008173
2020-08-14,0.008014,0.023244,NaN,0.005359,0.012370,0.019799,0.007030,0.001870,0.015935,1.192594e-46,...,0.014598,0.014894,0.009858,0.007856,0.051476,0.010695,0.010786,0.015487,0.014273,0.014346


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-12      0.014977  0.012374  0.003830  0.015733  0.010676  0.021068   
2020-08-13      0.010856  0.010769  0.006218  0.012281  0.011399  0.011550   
2020-08-14      0.005428  0.011661  0.006522  0.014893  0.013484  0.006789   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-12      0.003976  0.008320  0.002265  0.009512  ...  0.002866   
2020-08-13      0.003282  0.010998  0.001132  0.020885  ...  0.016248   
2020-08-14      0.004221  0.009277  0.002925  0.019123  ...  0.026109   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-08-12      5.960471e-08  0.002327  0.012929   0.006156  0.002274   
2020-08-13      1.000000e-01  0.008013  0.006465   0.004958  0.003818   
2020-08-14      5.000001e-02  0.031033  0.003232   0.009970  0.012576   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-08-12      0.000763       -1.0  0.035467 -0.016665  
2020-08-13      0.000382       -1.0  0.042124 -0.008332  
2020-08-14     -0.001608       -1.0  0.079201 -0.004166  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-12,0.019709,0.020536,NaN,0.021663,0.015207,0.014664,0.002239,0.000775,0.001093,NaN,...,0.017169,0.019763,0.009744,0.000138,0.000981,0.004053,0.006459,0.038152,0.005129,0.009545
2020-08-13,0.011980,0.010268,NaN,0.014972,0.016345,0.012772,0.002986,0.000387,0.001391,NaN,...,0.017894,0.022687,0.007721,0.000069,0.000490,0.004365,0.006710,0.019076,0.006027,0.022014
2020-08-14,0.006784,0.005134,NaN,0.012049,0.012468,0.019174,0.003087,0.000531,0.000695,NaN,...,0.013897,0.027053,0.013775,0.000035,0.000245,0.003664,0.008827,0.022610,0.006451,0.011007


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-12      0.012100  0.044042  0.050015  0.009359  0.072978   
2020-08-13      0.006050  0.022021  0.025008  0.004679  0.036489   
2020-08-14      0.003025  0.011011  0.012504  0.002340  0.018244   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-12      2.384196e-08  0.000893  0.075089  2.981260e-07  0.017857  ...   
2020-08-13      1.192098e-08  0.000447  0.037545  1.490630e-07  0.008929  ...   
2020-08-14      5.960489e-09  0.000223  0.018772  7.453150e-08  0.004464  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-12         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-13         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-14         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-12       0.009900      NaN    NaN  
2020-08-13       0.022807      NaN    NaN  
2020-08-14       0.011404      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200814,AK,4869,291060,NaN,38.0,NaN,NaN,NaN,3.0,...,295929,0,0,b2cce1fdb196ff1fca971a2c840e4d235b133307,0,0,0,0,0,NaN
1,20200814,AL,106309,718675,NaN,1376.0,12456.0,NaN,1305.0,NaN,...,824984,3,0,e21cf8a1da87dbadde9abd27c400ca2ec6c03a01,0,0,0,0,0,NaN
2,20200814,AR,52392,536268,NaN,466.0,3566.0,NaN,NaN,113.0,...,588660,5,0,bfb9ad7879f0715956511f23be1ae0d827c6e5bc,0,0,0,0,0,NaN
3,20200814,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,d511afb9b6dd6cb6c8d8633e39346e4cf227c73b,0,0,0,0,0,NaN
4,20200814,AZ,191721,875854,NaN,1359.0,20515.0,473.0,NaN,324.0,...,1067575,40,12,6c973bffe78189b38fcfe085b87faa693eefd530,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200814,AK,4869.0,291060.0,NaN,38.0,NaN,NaN,NaN,3.0,...,295929,0,0,b2cce1fdb196ff1fca971a2c840e4d235b133307,0,0,0,0,0,NaN
1,20200814,AL,106309.0,718675.0,NaN,1376.0,12456.0,NaN,1305.0,NaN,...,824984,3,0,e21cf8a1da87dbadde9abd27c400ca2ec6c03a01,0,0,0,0,0,NaN
2,20200814,AR,52392.0,536268.0,NaN,466.0,3566.0,NaN,NaN,113.0,...,588660,5,0,bfb9ad7879f0715956511f23be1ae0d827c6e5bc,0,0,0,0,0,NaN
3,20200814,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,d511afb9b6dd6cb6c8d8633e39346e4cf227c73b,0,0,0,0,0,NaN
4,20200814,AZ,191721.0,875854.0,NaN,1359.0,20515.0,473.0,NaN,324.0,...,1067575,40,12,6c973bffe78189b38fcfe085b87faa693eefd530,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-14,AK,4869,291060,NaN,38.0,NaN,NaN,NaN,3.0,NaN,...,295929,0,0,b2cce1fdb196ff1fca971a2c840e4d235b133307,0,0,0,0,0,NaN
2020-08-14,AL,106309,718675,NaN,1376.0,12456.0,NaN,1305.0,NaN,716.0,...,824984,3,0,e21cf8a1da87dbadde9abd27c400ca2ec6c03a01,0,0,0,0,0,NaN
2020-08-14,AR,52392,536268,NaN,466.0,3566.0,NaN,NaN,113.0,466.0,...,588660,5,0,bfb9ad7879f0715956511f23be1ae0d827c6e5bc,0,0,0,0,0,NaN
2020-08-14,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,d511afb9b6dd6cb6c8d8633e39346e4cf227c73b,0,0,0,0,0,NaN
2020-08-14,AZ,191721,875854,NaN,1359.0,20515.0,473.0,NaN,324.0,NaN,...,1067575,40,12,6c973bffe78189b38fcfe085b87faa693eefd530,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-14,AK,4869.0,291060.0,NaN,38.0,NaN,NaN,NaN,3.0,NaN,...,295929,0,0,b2cce1fdb196ff1fca971a2c840e4d235b133307,0,0,0,0,0,NaN
2020-08-14,AL,106309.0,718675.0,NaN,1376.0,12456.0,NaN,1305.0,NaN,716.0,...,824984,3,0,e21cf8a1da87dbadde9abd27c400ca2ec6c03a01,0,0,0,0,0,NaN
2020-08-14,AR,52392.0,536268.0,NaN,466.0,3566.0,NaN,NaN,113.0,466.0,...,588660,5,0,bfb9ad7879f0715956511f23be1ae0d827c6e5bc,0,0,0,0,0,NaN
2020-08-14,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,d511afb9b6dd6cb6c8d8633e39346e4cf227c73b,0,0,0,0,0,NaN
2020-08-14,AZ,191721.0,875854.0,NaN,1359.0,20515.0,473.0,NaN,324.0,NaN,...,1067575,40,12,6c973bffe78189b38fcfe085b87faa693eefd530,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE